## Clustering cells in different groups and visualize the enrichment of each category in each cluster


#### Steps:

- will be added (to do)
  

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(color_codes=True)
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from utils import read_data, visualize_data
from utils.read_data import *
from utils.visualize_data import *
from sklearn.cluster import KMeans
import time

In [ ]:
# ls /home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/workspace/drugSCprofiles/

### Input parameters

In [ ]:
images_dir="/home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/2016_04_01_a549_48hr_batch1_compressed/"
res_dir="./"

controlLabel='control'
drugLabel='drugX'


### Reading two single cell data dataframes for control and pertX
    * This is an example of two single cell dataframe, you may ignore below cell as you have your own inputs

In [ ]:
fileName="/home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/workspace/drugSCprofiles/BRD-A00147595-001-01-5_0.12.pkl"    
pair_dict = pickle.load( open(fileName, "rb" ) )
df_ss0=pair_dict['cp_ss']
pert_df_dmsos=pair_dict['cp_ss_control']

cp_features=pert_df_dmsos.columns[pert_df_dmsos.columns.str.contains("Cells_|Cytoplasm_|Nuclei_")].tolist()


cols2remove0=[i for i in df_ss0 if ((df_ss0[i]=='nan').sum(axis=0)/df_ss0.shape[0])>0.05]
cols2remove1=df_ss0[cp_features].std()[df_ss0[cp_features].std() < 0.00001].index.tolist()
cols2remove_drug=cols2remove0+cols2remove1    
print(cols2remove_drug)


cols2remove0=[i for i in pert_df_dmsos if ((df_ss0[i]=='nan').sum(axis=0)/pert_df_dmsos.shape[0])>0.05]
cols2remove1=pert_df_dmsos[cp_features].std()[pert_df_dmsos[cp_features].std() < 0.00001].index.tolist()
cols2remove_dmso=cols2remove0+cols2remove1        
print(cols2remove_dmso)


cols2remove=list(set(cols2remove_dmso+cols2remove_drug))
df_ss0=df_ss0.drop(cols2remove, axis=1);
pert_df_dmsos=pert_df_dmsos.drop(cols2remove, axis=1);

df_ss0=df_ss0.interpolate()
pert_df_dmsos=pert_df_dmsos.interpolate()

cp_features=pert_df_dmsos.columns[pert_df_dmsos.columns.str.contains("Cells_|Cytoplasm_|Nuclei_")].tolist()

### Creating "PathName_Orig"+channelX column for input dataframe containing the correct location of images

In [ ]:
images_dir="/home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/2016_04_01_a549_48hr_batch1_compressed/"

df_ss0["label"]=drugLabel
pert_df_dmsos["label"]="control"

wtANDmtDf=pd.concat([df_ss0,pert_df_dmsos], ignore_index=True)
channels=["Mito","AGP","DNA","RNA","ER"]
for ch in channels:
    wtANDmtDf["PathName_Orig"+ch]=images_dir+"images/"+wtANDmtDf["Metadata_Plate"]
    wtANDmtDf["FileName_Orig"+ch]=wtANDmtDf["FileName_Orig"+ch].apply(lambda x: x.replace("tiff","png"))

imgSize=wtANDmtDf.Metadata_ImageSizeX.values[0]
borderLength=int(np.percentile(wtANDmtDf.Cells_AreaShape_MajorAxisLength.values, 90)/2);    
wtANDmtDf=edgeCellFilter2(wtANDmtDf,imgSize,borderLength);  


### Apply Clustering Subpopulation analysis on input data
    And:
    - save the resulting images in "res_dir" path

In [ ]:
os.mkdir(res_dir+controlLabel+'-'+drugLabel)

#### sub pop analysis
wtANDmtDf_scaled = wtANDmtDf.copy()
scaler = preprocessing.RobustScaler()
locFeature2beremoved=list(filter(lambda x: "_Location_Center_X" in x or "_Location_Center_Y" in x, cp_features)) 
cpFeatures4scale=list(set(cp_features)-set(locFeature2beremoved))
wtANDmt=scaler.fit_transform(wtANDmtDf.loc[:,cpFeatures4scale].astype('float64'))
wtANDmtDf_scaled[cpFeatures4scale]=wtANDmt

nSampleSCs=6;boxSize=160;


nClus=8
kmeans = KMeans(n_clusters=nClus, random_state=0).fit(wtANDmtDf_scaled[cpFeatures4scale].values)
clusterLabels=kmeans.labels_
distances=kmeans.fit_transform(wtANDmtDf_scaled[cpFeatures4scale].values)
wtANDmtDf_scaled['clusterLabels']=clusterLabels;
wtANDmtDf_scaled['dist2Mean']=np.min(distances,1);

DirsDict={}
DirsDict['root']=images_dir
DirsDict['resDir']=res_dir+controlLabel+'-'+drugLabel

clusteringHists(DirsDict,wtANDmtDf_scaled,controlLabel,drugLabel,nClus,cpFeatures4scale,channels,boxSize)
